In [85]:
from elasticsearch import Elasticsearch
import pandas as pd
import json
import time
import logging
from datetime import date
import datetime
import re
import sys
import os
import os.path

max_size = 3

#Input: mystery_object<type irrelevant> Object you want to print
#       spacer<string> Its the string of tabs to indent the print as you start getting into nested items
#       flag<string> That denotes what the previous recurse id'd this field as to add helpful characters for reading output
#Output:STDOUT Easily Human readable layout of the data structure
#Description: Recurse print is the helper function to easy print that handles interpreting the input and breaks it into nested 
#             calls of this function till it finds an item that isnt iteratable
#             "-": Denotes that an item is an element in an array
#            "*:": Denotes that an item is a key in a dictionary
#             "+": Denotes that an item is the field referenced by a dictionary key
#            "\t": Denotes that an item is nested one layer deeper
def recursive_print(mystery_object, spacer, flag):
    if type(mystery_object) is dict:
        for key in mystery_object.keys():
            if flag == 'listdict':
                recursive_print(key, spacer + '-' , "key")
            elif flag == 'dict2':
                recursive_print(key, spacer + '+' , "key")
            else:
                recursive_print(key, spacer, "key")
            if type(mystery_object[key]) is dict:
                recursive_print(mystery_object[key], '\t' + spacer, "dict2")
            elif type(mystery_object[key]) is list:
                recursive_print(mystery_object[key], spacer, "list")
            else:
                recursive_print(mystery_object[key], '\t' + spacer, "field")
    elif type(mystery_object) is list:
        for item in mystery_object:
            if type(item) is dict:
                recursive_print(item, '\t' + spacer, "listdict")
            elif type(item) is list:
                recursive_print(item, '\t' + spacer, "list")
            else:
                recursive_print(item, '\t' + spacer, "element")
    else:
        if flag == "key":
            print(spacer + str(mystery_object) + ":")
        elif flag == "element":
            print(spacer + "-" + str(mystery_object))
        elif flag == "field":
            print(spacer + "+" + str(mystery_object))
        else:
            print(spacer + str(mystery_object))
            
            
#Input:        item<type irrelevant> Object you want to print
#Output:       STDOUT Easily Human readable layout of the data structure
#Description:
#              "-": Denotes that an item is an element in an array
#              "*:": Denotes that an item is a key in a dictionary
#              "+": Denotes that an item is the field referenced by a dictionary key       
def easy_print(item):
    recursive_print(item, "", "")
    
#Input:    node of elastisearch
#          index of table to search
#          array of days to search through
#          list of queries to be compared by or statement to elastisearch
#          list of ips that are to be whitelisted and ignored
#          columns of fields to collect data on
#Output: Dictionary of [Deisred Columns][Recorded Values][Count of each Value] to a file
#Subordinate Functions: redux_search_day
def or_redux_search_multiple(es_node, index, days, query_list, ip_whitelist, columns):
    es = Elasticsearch(es_node, timeout=30000)
    print("Searching for Desired Fields")
    print("//STARTING SEARCH FOR PLEASE WAIT//")
    days_start_time = time.time()
    logging.getLogger('elasticsearch').setLevel(logging.ERROR)
    must = []
    or_query = []
    for clean in ip_whitelist:
        must.append({"query_string" : {"analyze_wildcard" : "true", "query" : clean}})
    for or_q in query_list:
        or_query.append({"query_string" : {"analyze_wildcard" : "true", "query" : or_q}})
    or_query_body = {"bool" : {"should" : or_query}}
    must.append(or_query_body)
    query_body = {"bool": {"must": must}}
    for day in days:
        hits = es.search(index=index + day,query=query_body,size=max_size)
    difference = time.time() - days_start_time
    print("Time to Complete Total Search;", difference, "SECS")
    
    
    
es_node = 'http://10.1.19.100:9200'
index = 'arkime_sessions3'
days = ['-220316']
query_list = ["source.ip : 172.30.30.1"]
ip_whitelist = txt2list("../Whitelist/source.ip_whitelist.txt")
columns = ['source.ip','destination.ip']
or_redux_search_multiple(es_node, index, days, query_list, ip_whitelist, columns)

Searching for Desired Fields
//STARTING SEARCH FOR PLEASE WAIT//
Time to Complete Total Search; 0.07979035377502441 SECS
5


/opt/conda/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [82]:
def recursive_print(mystery_object, spacer, flag):
    if type(mystery_object) is dict:
        for key in mystery_object.keys():
            if flag == 'listdict':
                recursive_print(key, spacer + '-' , "key")
            elif flag == 'dict2':
                recursive_print(key, spacer + '+' , "key")
            else:
                recursive_print(key, spacer, "key")
            if type(mystery_object[key]) is dict:
                recursive_print(mystery_object[key], '\t' + spacer, "dict2")
            elif type(mystery_object[key]) is list:
                recursive_print(mystery_object[key], spacer, "list")
            else:
                recursive_print(mystery_object[key], '\t' + spacer, "field")
    elif type(mystery_object) is list:
        for item in mystery_object:
            if type(item) is dict:
                recursive_print(item, '\t' + spacer, "listdict")
            elif type(item) is list:
                recursive_print(item, '\t' + spacer, "list")
            else:
                recursive_print(item, '\t' + spacer, "element")
    else:
        if flag == "key":
            print(spacer + str(mystery_object) + ":")
        elif flag == "element":
            print(spacer + "-" + str(mystery_object))
        elif flag == "field":
            print(spacer + "+" + str(mystery_object))
        else:
            print(spacer + str(mystery_object))
        
def easy_print(item):
    recursive_print(item, "", "")

item = ["thing",
        "thing1", 
        "thing2", 
        "thing3", 
        "thing4", 
        ["thing5", "thing6"], 
        {"key" : "field", 
         "key1" : ["field1", "field5", "field6"], 
         "key2" : {"key3" : "field3", "key4" : ["field5", {"field6" : "idk"}, "field7"]}},
        "final_thing"
]
easy_print(item)

	-thing
	-thing1
	-thing2
	-thing3
	-thing4
		-thing5
		-thing6
	-key:
		+field
	-key1:
		-field1
		-field5
		-field6
	-key2:
		+key3:
			+field3
		+key4:
			-field5
			-field6:
				+idk
			-field7
	-final_thing
